# Note book to run colmap

NOTE: containers/pods for Colmap and neuralangelo are seperate as they have conflicting dependencies.

In [20]:
# Imports
import gdown
import os
import validators
from neovoxel_bot import neovoxel_bot

### SLACK BOT:

In [21]:
# Create Slack bot:
bot = neovoxel_bot.NeoVoxelBot(channel='neovoxel-bot')

### Data:

Download the data from drive and update video path variable.

In [15]:
#Download the video from drive:

#URl from google drive.
VIDEO_URL = 'https://drive.google.com/file/d/1QL3YAR5EMOID4e7hfkdk1uJFUfiLg3_R/view?usp=drive_link'

if validators.url(VIDEO_URL):
    print("Valid URL")
    !gdown --fuzzy {VIDEO_URL}
else:
    print("Invalid URL")


Valid URL
Downloading...
From: https://drive.google.com/uc?id=1QL3YAR5EMOID4e7hfkdk1uJFUfiLg3_R
To: /home/ashjostan/bucu/gitRepos/neuralangelo/notebooks/video-rig.mp4
100%|██████████████████████████████████████| 31.4M/31.4M [00:02<00:00, 15.1MB/s]


In [16]:
# File path: --> copy the path to which the video was downloaded
video_path = '/home/ashjostan/bucu/gitRepos/neuralangelo/notebooks/video-rig.mp4'

## COLMAP processing

1. Populate variables
2. Execute colmap command 

In [17]:
# Variables:
    # SEQUENCE: your custom name for the video sequence.
    # PATH_TO_VIDEO: absolute/relative path to your video.
    # DOWNSAMPLE_RATE: temporal downsampling rate of video sequence (for extracting video frames).
    # SCENE_TYPE: can be one of  {outdoor,indoor,object}.

SEQUENCE = 'rig-SLACK-BOT-TEST' # Give a relevant name. Eg: Leg_AFO_30thMarch24, bottle_rig_15thMarch25.
PATH_TO_VIDEO = video_path
DOWNSAMPLE_RATE = '2' #Higher the downsample rate, lower number of frames extracted.
SCENE_TYPE = 'object' # currently most of our experiments are objects.

# Run colmap:

In [18]:
# FFMPEG:
if SEQUENCE == '': # To avoid weird path creation
    print('[INFO] Setting sequence name to default value.')
    SEQUENCE = 'video_sequence'

data_path=f'datasets/{SEQUENCE}_ds{DOWNSAMPLE_RATE}'
print(f'[INFO] File path: {PATH_TO_VIDEO}')
if os.path.exists(PATH_TO_VIDEO):
    !chmod +x ../projects/neuralangelo/scripts/run_ffmpeg.sh
    !bash ../projects/neuralangelo/scripts/run_ffmpeg.sh {SEQUENCE} {PATH_TO_VIDEO} {DOWNSAMPLE_RATE}
else:
    print(f'[ERROR]: Following video path does not exist {PATH_TO_VIDEO}')


[INFO] File path: /home/ashjostan/bucu/gitRepos/neuralangelo/notebooks/video-rig.mp4
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libthe

In [19]:
# Run COLMAP:
print(f'[INFO]: Path to dataset: {data_path}')
bot.send_message(f'*COLMAP*: processing _{SEQUENCE}_ started')
try:
    !bash ../projects/neuralangelo/scripts/run_colmap.sh {data_path}
except Exception as e:
    bot.send_message(f'*COLMAP*: processing error:_{e}_')
bot.send_message(f'*COLMAP*: completed {SEQUENCE}')

[INFO]: Path to dataset: datasets/rig-SLACK-BOT-TEST_ds2
I0324 13:01:30.635879 13682 misc.cc:44] 
Feature extraction
I0324 13:01:30.638213 13715 sift.cc:721] Creating SIFT GPU feature extractor
I0324 13:01:30.884088 13716 feature_extraction.cc:258] Processed file [1/1556]
I0324 13:01:30.884127 13716 feature_extraction.cc:261]   Name:            000001.jpg
I0324 13:01:30.884130 13716 feature_extraction.cc:270]   Dimensions:      720 x 1280
I0324 13:01:30.884135 13716 feature_extraction.cc:273]   Camera:          #1 - SIMPLE_RADIAL
I0324 13:01:30.884138 13716 feature_extraction.cc:276]   Focal Length:    1536.00px
I0324 13:01:30.884150 13716 feature_extraction.cc:280]   Features:        106
I0324 13:01:30.901194 13716 feature_extraction.cc:258] Processed file [2/1556]
I0324 13:01:30.901238 13716 feature_extraction.cc:261]   Name:            000002.jpg
I0324 13:01:30.901244 13716 feature_extraction.cc:270]   Dimensions:      720 x 1280
I0324 13:01:30.901247 13716 feature_extraction.cc:273

In [ ]:
# Generate transforms.json
!pip install numpy #Common error seen. - requires a pip install of numpy. 

#Git init submodule -> colmap repo:
!git submodule update --init --recursive

# One of the two commands works depending on machine:
# RUNPOD behaviour noticed: In runpod it picks up a different version of python and does not find compatible numpy. 
try:
    !python ../projects/neuralangelo/scripts/convert_data_to_json.py --data_dir {data_path} --scene_type {SCENE_TYPE}
except Exception as e:
    !python3.11 ../projects/neuralangelo/scripts/convert_data_to_json.py --data_dir {data_path} --scene_type {SCENE_TYPE}


### Send Data: RunpodCTL


!runpodctl send {data_path}